In [85]:
import requests
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
import random
#同意字
sysets = []
with open('./ss.txt' , 'r' , encoding='utf-8') as f :
    xs = f.readlines()
    for s in xs:
        s = s.split(',')
        s.pop()
        sysets.append(s)
        
columns =['keywords','地點','工作抬頭','公司名','工作url','需要工作經歷','需求科系','語言','語言程度']
skill=[]
skill_bigdata=[]
data = []
url = f'https://www.104.com.tw/jobs/search/'
keywords = []
with open('./bigData.txt','r',encoding='utf-8-sig') as f :
    for x in f.readlines():
        keywords.append(x.replace('\n',''))
# keywords = ['資料分析','數據分析']
for keyword in keywords:
    print(f'---------{keyword}職位---------')
    for i in range(3):
        params={'keyword':keyword,'page':i}
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
        res = requests.get(url=url , headers=headers , params=params)
        soup = BeautifulSoup(res.text , 'html.parser')
        articles = soup.select('article.b-block--top-bord')
        for article in articles:
            try:
                time.sleep(random.uniform(0,2))
                job_data = []
                if len(article.select('svg.b-icon--gray.b-icon--w18 use')) == 3:
                    continue
                job_title=article.select('a.js-job-link')[0].text #工作抬頭
                job_url ='https:'+article.select('a.js-job-link')[0]['href'] #工作url
                company=article.select('ul.b-list-inline li a')[0].text.replace(' ','').replace('\n','') #公司
                print(company)
                job_address=article.select('ul.b-list-inline.b-clearfix.job-list-intro li')[0].text #地區
                ajax_url_x  = re.search('/.{5}\?' , job_url).span()
                x = job_url[(ajax_url_x[0]+1):(ajax_url_x[1]-1)]
                ajax_url=f'https://www.104.com.tw/job/ajax/content/{x}'
                headers['Referer']=f'https://www.104.com.tw/job/{x}'
                job_res =requests.get(url =ajax_url,headers=headers)
                jd=json.loads(job_res.text)
                job_skills=jd['data']['condition']['specialty'] #需要技能
                job_workexp=jd['data']['condition']['workExp']  #工作經歷
                 #學歷
                if len(jd['data']['condition']['major'])==0:
                    job_major=None
                else:
                    job_major=jd['data']['condition']['major']

                #語言
                job_language=jd['data']['condition']['language']
                if len(job_language) > 0 :
                    job_language_l = jd['data']['condition']['language'][0]['language']
                    job_language_level = jd['data']['condition']['language'][0]['ability']
                    job_language_data = job_language_l+':'+job_language_level
                else:
                    job_language_l = None
                    job_language_level = None     
                job_data=[keyword,job_address,job_title,company,job_url,job_workexp,job_major,job_language_l,job_language_level]
                skill_data=[]
                for job_skill in job_skills:
                    y = 0
                    for syset in sysets:
                        if job_skill['description'] in syset: 
                            skill.append(syset[0])
                            skill_data.append(syset[0])
                            y = y+1
                            break
                    if y == 0 :
                        skill.append(job_skill['description'])
                        skill_data.append(job_skill['description'])
                print(skill_data)
                data.append(job_data)
                skill_bigdata.append(skill_data)
            except AttributeError:
                print('資料有誤')
                
#     time.sleep(2)
skill=set(skill)
# columns =['keyword','地點','工作抬頭','公司名','工作url','需要工作經歷','需求科系','語言','語言程度']
df = pd.DataFrame(data=data , columns=columns)
for s in skill:
    df[s]=''
for i , xs in enumerate(skill_bigdata):
    if len(s) == 0:
        continue
    for s in xs:
        df[s][i]='v'

df.to_csv('./104data.csv',encoding='utf-8-sig')
df.head()

---------資料分析職位---------
精藤股份有限公司
[]
中國信託商業銀行股份有限公司
['Python', 'SQL']
國立台灣大學_中國信託慈善基金會兒少暨家庭研究中心
['R', '基礎文書處理', 'Outlook', '基礎文書處理', '基礎文書處理', 'SAS', 'SPSS']
亞洲指標數位行銷顧問股份有限公司
['Python', 'R']
邑泰科技股份有限公司
['Python', 'ETL', 'SQL', 'SQL', 'Linux']
百歐生命科技股份有限公司
[]
國票綜合證券股份有限公司
['SPSS']
寶勝國際(控股)_寶原興業股份有限公司
['Python', 'SQL', '基礎文書處理', '基礎文書處理', '基礎文書處理']
微星科技股份有限公司
['Python', 'HTML', 'jQuery', 'Tableau', 'Power BI\u200b']
關貿網路股份有限公司
['Python', 'R', 'SAS', 'SPSS']
康乃爾英語股份有限公司
['Python', 'R', '基礎文書處理', '基礎文書處理', 'SPSS']
果實夥伴股份有限公司
['Python', 'SQL']
萬寶華企業管理顧問股份有限公司
['SQL', 'SAS']
泰詠電子股份有限公司
['Python', '基礎文書處理']
廣晉軟體股份有限公司
['SQL', 'SQL', 'Tableau', 'SAS']
智聯服務股份有限公司
['SQL', '基礎文書處理']
智聯服務股份有限公司
['Linux', 'Python', 'SQL', 'Linux', '基礎文書處理']
立樂高園股份有限公司
[]
台固媒體股份有限公司
['Linux', 'Git', 'PHP', 'Python', 'SQL', 'hadoop', 'JavaScript']
遠東百貨股份有限公司
['Python', 'SQL', '基礎文書處理', 'Tableau']
果實夥伴股份有限公司
['Python', 'SQL']
寶勝國際(控股)_寶原興業股份有限公司
['Python', 'SQL', '基礎文書處理', '基礎文書處理', '基礎文書處理']
精藤股份有限公司
[]
國立台灣大學_中國信託慈善

雨林新零售股份有限公司
['SQL', 'SQL', 'Linux', 'SQL', '基礎文書處理', '基礎文書處理', '基礎文書處理', 'Power BI\u200b']
干城數碼有限公司
['Power BI\u200b']
凌群電腦股份有限公司
['ETL', 'SQL', 'SQL']
樂天國際商業銀行股份有限公司
['ETL', 'SQL']
華卡企業股份有限公司
['ASP.NET', 'Java', 'Python', 'SQL', 'SQL']
德昂資訊股份有限公司
[]
美商和諧國際有限公司台灣分公司
[]
聯合通商電子商務股份有限公司
['SQL', 'SQL', 'Linux']
德昂資訊股份有限公司
[]
國泰金控_國泰金融控股股份有限公司
[]
國泰世紀產物保險股份有限公司
[]
聯合通商電子商務股份有限公司
['DataStage', 'ETL', 'Informatica', 'SQL', 'Linux']
明台產物保險股份有限公司_資訊部(三井住友海上集團)
['SQL', 'Linux', 'SQL']
歐立威科技股份有限公司
['Linux']
德昂資訊股份有限公司
['SQL', 'Data Architect', 'Data Marts', 'MRP', 'ERP', 'ERP', 'ERP', 'SAP', 'ERP', 'ERP', 'ERP', '文中系統', 'ERP', 'Microsoft Dynamics AX', 'ERP']
中信金控_台灣人壽保險股份有限公司
['Shell', 'ETL', 'Informatica']
誠品股份有限公司
['C#', 'SQL', 'SQL']
國泰世華商業銀行股份有限公司_人力資源部
['Linux', 'ASP.NET', 'Java', 'Cognos', 'Informatica', 'Linux', 'Tableau']
樂天國際商業銀行股份有限公司
['ETL', 'SQL']
智聯服務股份有限公司
['Linux', 'ASP.NET', 'Java', 'Python', 'ETL', 'SQL']
Garena_新加坡商競舞電競有限公司臺灣分公司
['Linux', 'Java', 'Python', 'Scala', 'hadoop', 'Hi

欣創科技有限公司
['Python']
日新軟體股份有限公司
['Python', 'HTML', 'JavaScript', 'jQuery', 'CSS', 'Django']
禾向數位科技有限公司
['Linux', 'Python', 'SQL']
StreetVoice_街聲股份有限公司
['Python']
仰得股份有限公司
['Python']
瞬聯科技股份有限公司
['Python']
安智聯科技有限公司
['Linux', 'OOP', 'UML', 'Git', 'Git', 'Python', 'SQL', 'SQL']
仰得股份有限公司
['Python']
博思資訊管理顧問有限公司
['Python']
BigGo_樂方股份有限公司
['Linux', 'Python', 'Go', 'JavaScript']
緯德科技股份有限公司(TechLinkCorporation)
['Python']
台灣宸運雲端有限公司
['Linux', 'Python']
日新軟體股份有限公司
['Python', 'HTML', 'JavaScript', 'jQuery', 'CSS', 'Django']
禾向數位科技有限公司
['Linux', 'Python', 'SQL']
學力創意股份有限公司
['Python']
臺灣發展軟體科技股份有限公司
['Python']
米約科技有限公司
['Linux', 'Python', 'SQL', 'SQL']
固特斯股份有限公司
['Linux', 'Python']
博賢智能科技有限公司
['Python', 'JavaScript']
春水堂科技娛樂股份有限公司
['Linux', 'Python', 'SQL', 'jQuery']
普鴻資訊股份有限公司
['Python']
文境資科股份有限公司
['Shell', 'Python']
創奕能源科技股份有限公司
['Python']
BigGo_樂方股份有限公司
['Linux', 'Python']
橋新科技股份有限公司
['Python', 'R']
欣創科技有限公司
['Python']
祐安資訊有限公司
['Python']
干城數碼有限公司
['Python', 'SQL', 'SQL']
宏佳騰動力科技股份有限公司
['Python

悠活水產股份有限公司
['PHP', 'SQL', 'SQL']
全日物流股份有限公司
['Linux', 'SQL']
尊博科技股份有限公司
['Database Administrator', 'Database Management', 'SQL']
大極科技有限公司
[]
堯晉科技有限公司
[]
九比一股份有限公司
[]
瑞嘉軟體科技股份有限公司
[]
SpringProfessional_藝珂人事顧問股份有限公司躍科分公司
[]
易訂網股份有限公司
['Linux', 'PHP', 'SQL']
睿訊有限公司
['Linux', 'Shell', 'UNIX', 'SQL', 'SQL', 'Linux', 'SQL', 'Vmware']
勝圖國際企業股份有限公司
['SQL']
耀聖資訊科技股份有限公司
['C#', 'ADO', 'SQL', 'SQL']
馬紹爾群島商彩虹科技股份有限公司
['Linux', 'C++', 'SQL']
馬紹爾群島商彩虹科技股份有限公司
['Linux', 'C++', 'SQL']
馬紹爾群島商彩虹科技股份有限公司
['Linux', 'PHP', 'SQL']
環球誠信有限公司
['Python', 'SQL', 'Django']
拾雲科技有限公司
['SQL']
年豐企業有限公司
['ASP.NET', 'C#', 'SQL', 'SQL']
拾雲科技有限公司
['SQL']
皆凱科技有限公司
['Linux', 'SQL']
大牛科技有限公司
['Linux', 'Shell', 'JDBC', 'SQL', 'OLAP', 'SQL']
傑西樓口智能科技股份有限公司
['AJAX', 'ASP.NET', 'C#', 'PHP', 'Python', 'SQL', 'HTML', 'JavaScript', 'CSS']
中鋼集團_興達海洋基礎股份有限公司
['Java', 'SQL', 'HTML', 'CSS']
一通數位有限公司
['Linux', 'Shell', 'Python', 'SQL', 'Linux']
伊歐網路科技有限公司
['Linux', 'Git', 'SQL', 'DNS', 'HTTP']
友上科技股份有限公司
['Python', 'SQL', 'Django']
嘉宇科

英商引達取有限公司台灣分公司
['Linux', 'Python', 'Data Modeling', 'ETL', 'SQL', 'SQL', 'OLAP', 'Linux', 'SQL', 'hadoop', 'JavaScript', 'Data Architect', 'Data Marts', 'Hive']
國泰世華商業銀行股份有限公司_人力資源部
['ETL', 'hadoop', 'Hive']
炬識科技股份有限公司
[]
香港商聲吶天空資訊顧問有限公司
[]
Garena_新加坡商競舞電競有限公司臺灣分公司
['Linux', 'Java', 'Python', 'Scala', 'hadoop', 'Hive']
國泰世華商業銀行股份有限公司_人力資源部
['Python', 'SQL', 'hadoop', 'SNA', 'Hive']
和碩集團_和碩聯合科技股份有限公司
['Linux', 'Java', 'Python', 'ETL', 'SQL', 'hadoop', 'HTTP']
威朋大數據股份有限公司
[]
萬寶華企業管理顧問股份有限公司
['Java', 'Go', 'hadoop', 'Hive']
台北神策諮詢顧問有限公司
['Java', 'SQL', 'hadoop', 'Hive']
台灣樂天市場股份有限公司
['Java', 'Python', 'SQL', 'Django', 'Hive']
ShoplineLimited_商線科技股份有限公司
[]
國泰世華商業銀行股份有限公司_人力資源部
['Linux', 'Shell', 'Python', 'ETL', 'hadoop', 'Hive']
英諾瓦資訊科技服務有限公司
[]
酷遊天股份有限公司
[]
願境網訊股份有限公司_KKBOX
[]
銓鍇國際股份有限公司
[]
艾酷互動股份有限公司
['Linux', 'Git', 'Python', 'SQL']
達暉資訊有限公司
['Python', 'ETL']
紅石資訊有限公司
['Python', 'R', 'hadoop', 'Hive']
英商引達取有限公司台灣分公司
['Linux', 'Python', 'Data Modeling', 'ETL', 'SQL', 'SQL', 'OLAP', 'Lin

...

速準科技股份有限公司
['AutoCAD', 'AutoCad 2D', 'AutoCad 3D', 'SolidWorks', 'CAM']
聯創數位工作室
['ASP.NET', 'Java', 'PHP', 'SQL', 'SQL', 'HTML', 'JavaScript', 'XML', 'jQuery', 'CSS', 'Adobe Photoshop']
振興發科技有限公司
['ASP.NET', 'C#', 'SQL', 'HTML', 'JavaScript']
高雄市立聯合醫院
['Windows ', 'Windows ', 'Windows ', 'ASP.NET', 'Visual Basic .net', 'Visual C#', 'SQL', 'HTML', 'JavaScript', 'XML Web services', 'jQuery', 'CSS']
筌鴻科技股份有限公司
['AJAX', 'ASP.NET', 'C#', 'HTML', 'JavaScript', 'jQuery', 'CSS']
嘉宇科技股份有限公司
['Linux', 'PHP', 'SQL', 'HTML', 'JavaScript', 'jQuery']
協正金屬實業有限公司
[]
國立中山大學_人事室
['C++', 'PHP', 'SQL']
五方科技股份有限公司
['AJAX', 'ASP.NET', 'JSP', 'SQL', 'Linux', 'SQL']
宏偉電機工業股份有限公司
['Delphi', 'SQL']
興普科技股份有限公司
['ASP.NET', 'C#', 'SQL', 'JavaScript']
豐盛鎂合金股份有限公司
[]
翔睿德股份有限公司
['AJAX', 'ASP.NET', 'SQL', 'HTML', 'JavaScript', 'jQuery', 'CSS', 'JavaScript']
恆煦資訊股份有限公司
['OOP', 'AJAX', 'SQL', 'HTML', 'HTML', 'JavaScript', 'jQuery', 'JavaScript', 'JavaScript']
藍科數位科技股份有限公司
資料有誤
振興發科技有限公司
['ASP.NET', 'C#', 'SQL']
總泰技術整合有限

雙子數位科技有限公司
['ETL', 'Informatica', 'SQL', 'Linux']
美商網基股份有限公司台灣分公司
[]
艾力特科技有限公司
['SAP']
台灣大車隊股份有限公司
['Java', 'Python', 'SQL', 'Tableau']
歐立威科技股份有限公司
['Linux', 'ETL']
達暉資訊有限公司
['Python', 'ETL']
緯德科技股份有限公司(TechLinkCorporation)
['DataStage', 'ETL', 'Informatica', 'SQL', 'SQL', 'Linux', 'SQL']
凌群電腦股份有限公司
['ETL', 'SQL', 'SQL']
緯德科技股份有限公司(TechLinkCorporation)
['DataStage', 'ETL', 'Informatica', 'SQL', 'SQL', 'Linux', 'SQL']
智聯服務股份有限公司
['Linux', 'ASP.NET', 'Java', 'Python', 'ETL', 'SQL']
聯合通商電子商務股份有限公司
['DataStage', 'ETL', 'Informatica', 'SQL', 'Linux']
邑泰科技股份有限公司
['Python', 'ETL', 'SQL', 'SQL', 'Linux']
樂天國際商業銀行股份有限公司
['ETL', 'SQL']
德昂資訊股份有限公司
['SQL', 'Data Architect', 'Data Marts', 'MRP', 'ERP', 'ERP', 'ERP', 'SAP', 'ERP', 'ERP', 'ERP', '文中系統', 'ERP', 'Microsoft Dynamics AX', 'ERP']
圖策科技股份有限公司
['Python', 'ETL']
台灣析數資訊股份有限公司
['ETL', 'SQL']
歐立威科技股份有限公司
['Linux', 'ETL']
樂天國際商業銀行股份有限公司
['ETL', 'SQL']
邑泰科技股份有限公司
['Python', 'ETL', 'SQL', 'SQL', 'Linux']
德義資訊股份有限公司
['Java', 'JSP', 'ETL', 'SQL', 'OL

大樹連鎖藥局_大樹醫藥股份有限公司
['DataStage', 'ETL', 'Informatica', 'SQL', 'SQL', 'Linux', 'SQL']
明台產物保險股份有限公司_資訊部(三井住友海上集團)
['Linux', 'SQL']
布魯科技有限公司
['SQL']
台灣經濟新報文化事業股份有限公司
['Java', 'SQL', 'SQL', 'JavaScript']
祐安資訊有限公司
['DataStage', 'ETL', 'Informatica', 'SQL', 'SQL', 'Linux', 'SQL']
北祥科技服務股份有限公司
[]
明台產物保險股份有限公司(三井住友海上集團)
['Linux', 'SQL']
東森購物_電視購物_東森得易購股份有限公司
[]
新加坡商齊舵管理顧問有限公司台灣分公司
['Java', 'SQL']
欣創科技有限公司
['DataStage', 'ETL', 'Informatica', 'SQL', 'SQL', 'Linux', 'SQL']
瑞嘉軟體科技股份有限公司
[]
科萊博瑞科技有限公司
['SQL']
群益金鼎證券股份有限公司
['C#', 'SQL', 'HTML', 'JavaScript']
鼎恒數位科技股份有限公司
['SQL', 'SQL']
台灣應用智能系統有限公司
['SQL', '基礎文書處理', '基礎文書處理', 'Visio', '基礎文書處理']
萬寶華企業管理顧問股份有限公司
['VBA', 'SQL']
群益金鼎證券股份有限公司
['ASP.NET', 'C#', 'SQL', 'HTML', 'jQuery']
藝珂人事顧問股份有限公司
['SQL', 'SAS']
和運租車股份有限公司
[]
明台產物保險股份有限公司(三井住友海上集團)
['Linux', 'SQL']
艾力特科技有限公司
[]
巨耀資訊顧問有限公司
['SQL', 'SQL', 'SQL']
德克互聯科技有限公司
['SQL', 'SQL']
欣創科技有限公司
['DataStage', 'ETL', 'Informatica', 'SQL', 'SQL', 'Linux', 'SQL']
關東鑫林集團桃園廠_義鎧科技股份有限公司
['SQL']
台灣瑞肯物流股份有限公司
['Da

美時化學製藥股份有限公司
[]
德商默克在台集團_台灣賽孚思科技股份有限公司
[]
IMECTAIWANCO.比利時微電子研究中心_台灣愛美科股份有限公司
['ARM', 'Circuit Design', 'FPGA', 'RTL', 'Verilog']
香港商希瑞科技股份有限公司台灣分公司
['Linux', 'Git', 'C++', 'Python', 'SAN/NAS']
安霸股份有限公司
['Perl', 'TCL', 'ASIC', 'EDA', 'Verilog']
梭特科技股份有限公司
[]
銳鋒股份有限公司
['Matlab', 'Python', 'RF']
安成國際藥業股份有限公司
['基礎文書處理', 'Outlook', '基礎文書處理', '基礎文書處理']
安成國際藥業股份有限公司
['基礎文書處理', 'Outlook', '基礎文書處理', '基礎文書處理']
光聚合有限公司
[]
美商英特格有限公司台灣分公司_半導體零件商
[]
博賢人力資源有限公司
[]
台灣杜邦股份有限公司
[]
緯創軟體股份有限公司
['PHP']
愛德萬測試股份有限公司
['AutoCAD', 'Pro/E', 'SolidWorks']


,keywords,地點,工作抬頭,公司名,工作url,需要工作經歷,需求科系,語言,語言程度,Rails,...,Win32,Git,SWIFT,Hive,XSL,ArcGis,Sniffer,SQL,Systems Administration,CAM
0,資料分析,台北市中山區,資料分析師,精藤股份有限公司,https://www.104.com.tw/job/4oqww?jobsource=n10...,2年以上,None,None,None,,...,,,,,,,,,,
1,資料分析,台北市南港區,洗錢防制_數據資料分析人員,中國信託商業銀行股份有限公司,https://www.104.com.tw/job/710wp?jobsource=n10...,3年以上,None,英文,聽 /精通、說 /精通、讀 /精通、寫 /精通,,...,,,,,,,,v,,
2,資料分析,台北市大安區,研究員/博士後研究員(兒少保資料分析),國立台灣大學_中國信託慈善基金會兒少暨家庭研究中心,https://www.104.com.tw/job/5um6g?jobsource=n10...,不拘,"[社會學相關, 其他經社心理相關, 教育學科類]",英文,聽 /中等、說 /中等、讀 /中等、寫 /中等,,...,,,,,,,,,,
3,資料分析,台北市松山區,資料分析師 (Data Analyst),亞洲指標數位行銷顧問股份有限公司,https://www.104.com.tw/job/6k2x6?jobsource=n10...,不拘,[數學及電算機科學學科類],英文,聽 /中等、說 /中等、讀 /精通、寫 /精通,,...,,,,,,,,,,
4,資料分析,台北市內湖區,資料分析師,邑泰科技股份有限公司,https://www.104.com.tw/job/3y0pq?jobsource=n10...,3年以上,None,None,None,,...,,,,,,,,v,,


In [81]:
syset = []
with open('./ss.txt' , 'r' , encoding='utf-8') as f :
    xs = f.readlines()
    for s in xs:
        s = s.split(',')
        s.pop()
        syset.append(s)
# print(syset)
for xs in syset:
        if 'Windows 7' in xs:
            print('good')

good
